In [ ]:
import os
import dotenv

dotenv.load_dotenv()

if not os.getenv("GITHUB_TOKEN"):
    raise ValueError("GITHUB_TOKEN is not set")

os.environ["OPENAI_API_KEY"] = os.getenv("GITHUB_TOKEN")
os.environ["OPENAI_BASE_URL"] = "https://models.inference.ai.azure.com/"

In [ ]:
from llama_index.llms.openai import OpenAI
from llama_index.embeddings.openai import OpenAIEmbedding
from llama_index.core import VectorStoreIndex, SimpleDirectoryReader, StorageContext, load_index_from_storage
from llama_index.core import Settings
import os

llm = OpenAI(
    model="gpt-4o-mini",
    api_key=os.getenv("OPENAI_API_KEY"),
    api_base=os.getenv("OPENAI_BASE_URL"),
)

embed_model = OpenAIEmbedding(
    model="text-embedding-3-small",
    api_key=os.getenv("OPENAI_API_KEY"),
    api_base=os.getenv("OPENAI_BASE_URL"),
)

Settings.embed_model = embed_model

In [ ]:
import phoenix as px
px.launch_app()

In [ ]:
from openinference.instrumentation.llama_index import LlamaIndexInstrumentor
from phoenix.otel import register

tracer_provider = register()
LlamaIndexInstrumentor().instrument(tracer_provider=tracer_provider)

In [ ]:
# change chunk size and chunk overlap via Settings

In [ ]:
try:
    storage_context = StorageContext.from_defaults(
        persist_dir="../local_index_chunked_xyz" # make sure you change this name
    )
    index = load_index_from_storage(storage_context)
except:
    #Note: we have to reduce the batch size to stay within the token limits of the free service
    documents = SimpleDirectoryReader("../Big Star Collectibles").load_data()
    index = VectorStoreIndex.from_documents(documents, insert_batch_size=150)
    index.storage_context.persist(persist_dir="../local_index_chunked_xyz")

In [ ]:
# solution
query_engine = index.as_query_engine(
  llm=llm
)
query_engine.query("When did the story for Big Star Collectibles Start?")

In [ ]:
query_engine.query("Who started Big Star Collectibles?")